# **Effective Model**

In [1]:
import cmath
import numpy as np
import matplotlib.pyplot as plt

from MTI_Second_Order import FMTI2_NeumannBC, Change_Basis

# labels for curves
Flabels = [["$f_{\\uparrow +, \\uparrow +}$", "$f_{\\uparrow +, \downarrow +}$", "$f_{\\uparrow +, \\uparrow -}$", "$f_{\\uparrow +, \downarrow -}$"],
          ["$f_{\downarrow +, \\uparrow +}$", "$f_{\downarrow +, \downarrow +}$", "$f_{\downarrow +, \\uparrow -}$", "$f_{\downarrow +, \downarrow -}$"],
          ["$f_{\\uparrow -, \\uparrow +}$", "$f_{\\uparrow -, \downarrow +}$", "$f_{\\uparrow -, \\uparrow -}$", "$f_{\\uparrow -, \downarrow -}$"],
          ["$f_{\downarrow -, \\uparrow +}$", "$f_{\downarrow -, \downarrow +}$", "$f_{\downarrow -, \\uparrow -}$", "$f_{\downarrow -, \downarrow -}$"]]

# colors for plotting
colors_real = ['brown', 'red', 'orange', 'gold']
# colors for plotting
colors_imag = ['blue','dodgerblue','lightskyblue','cyan']
# colors for plotting
colors_abs = ["#0077FF", "#00CC00", "#FFD700", "#FF2200"]

## **Channel Decomposition** 

We decompose the induced pairing in a basis of 16 matrices that generates the whole space made by the outer product of spin $\sigma = \uparrow, \downarrow$ and orbital $\lambda=\pm$.
The $4 \times 4$ matrix can be decomposed as
$$
    \Delta_{ind} (\mathbf{k},\omega) = \sum_A f_A (\mathbf{k},\omega) \Lambda_A \,,
$$
where the coefficients of the expansion can be computed through
$$
    f_A (\mathbf{k},\omega) = 
    \frac{ \text{Tr}\left( \Lambda_A^\dagger \, \Delta_{ind} (\mathbf{k},\omega)  \right) }
    { \text{Tr}\left( \Lambda_A^\dagger \, \Lambda_A  \right) }
$$and $\{ \Lambda_A \}$ is a basis of 16 matrices that generates the whole space.
We choose the following basis in spin space
$$
    \left\{ S_a \right\} = 
    \left\{ 
    \frac12 (\sigma_0+\sigma_z), \frac12 (\sigma_0-\sigma_z), \frac{1}{\sqrt2} \sigma_x, \frac{i}{\sqrt2} \sigma_y 
    \right\} \,,
$$
where $\sigma_{x,y,z}$ is the usual set of Pauli matrices acting in spin space.
The first three matrices of the basis correspond to symmetric spin triplet channels $f_{\uparrow\uparrow}, f_{\downarrow\downarrow}$ and $\frac{1}{\sqrt2}(f_{\uparrow\downarrow} + f_{\downarrow\uparrow})$, while the last one correspond to the antisymmetric spin-single channel $\frac{1}{\sqrt2}(f_{\uparrow\downarrow} - f_{\downarrow\uparrow})$.
Choosing a similar basis for the orbital part 
$$
    \left\{ L_a \right\} = 
    \left\{ 
    \frac12 (\lambda_0+\lambda_z), \frac12 (\lambda_0-\lambda_z), \frac{1}{\sqrt2} \lambda_x, \frac{i}{\sqrt2} \lambda_y 
    \right\} \,,
$$
the full set of matrices $\{ \Lambda_A \}$ can be obtained as
$$
    \Lambda_A = S_a \otimes L_\alpha \,,
$$
for $a \in \{ \uparrow\uparrow, \downarrow\downarrow, \text{sym}, \text{asym} \}$ and $\alpha \in \{ ++, --, \text{sym}, \text{asym} \}$, where $\text{sym}$ ($\text{asym}$) label the symmetric (antisymmetric) mixed state.

#### <font color='green'> **Render Channels** </font> 

In [2]:
from sympy import Matrix, Symbol
from Pairing_Effective_Model import Render_Projection, Render_Channel, Pairing_Projection, Pairing_Channel

# define your physical basis labels (spin x orbital)
basis = [r"\uparrow+", r"\uparrow-", r"\downarrow+", r"\downarrow-"]

# build a 4×4 Matrix of Symbols f_{α,β}
M = Matrix([[ Symbol(f"f_{{{i},{j}}}") for j in basis ] for i in basis ])

# spin and ordital indices
s = 's0'; o = 't0'

# projection 
Render_Projection(M=M, spin=s, orbital=o)
# pairing channel
Render_Channel(spin=s, orbital=o)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### <font color='green'> **Projected Components** </font> 

In [3]:
# finite z-lattice
d = 100.; z = 25
# parameters
mu = 0.010; Delta = 0.005; L = 0.020; omega = 0.0025
# coupling matrices
Gamma = np.matrix([[1,0],[0,1],[1,0],[0,1]])

# compute F at given parameters
F2 = Change_Basis(FMTI2_NeumannBC(d=d, z=z, Z=z, z0=0, kx=0.01, ky=0., L=L, mu=mu, Delta=Delta, omega=omega, Gamma=Gamma))

s = 's0'; o = 't0'

proj = Pairing_Projection(Delta=F2, spin=s, orbital=o)
print()
print(proj)
print()

# get the matrix corresponding to projection
c = Pairing_Channel(spin=s, orbital=o)
print(c)

trace =  (4+0j)

(0.5997315877468452-0.0034443538586635125j)

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


### **Induced Pairing**

In [4]:
# shape of the matrix 
n = 4

# space dimensions
K = []; Z = []
k_dim = 100; z_dim = 100

# read file pairing
rows = np.loadtxt("Pairing/pairing_L-p20.txt")

# store data in matrices
F_data = np.empty((k_dim, z_dim, 4, 4), dtype='complex')


# loop over momenta
for idx in range(k_dim):
    
    # loop over space
    for jdx in range(z_dim):

        # index for rows
        irow = z_dim * idx + jdx
                
        # read one row
        row = rows[irow]
    
        if jdx == 0: K.append(float(row[0]))
    
        if idx == 0: Z.append(float(row[1]))
    
        # real part
        real = row[2 : 2 + n*n].reshape((n, n))
        # imaginary part
        imag = row[2 + n*n :].reshape((n, n))
        # pairing matrix
        matrix = real + 1j * imag

        F_data[idx][jdx] = Change_Basis(matrix)

print(F_data.shape)

(100, 100, 4, 4)
